# CSCI 3155 Recitation 11

November 30, 2018

## Class Heierarchy
The classes we'll be using for this recitation:

In [1]:
class Department

class Engineering extends Department
class CompSci extends Engineering
class MechE extends Engineering

class Humanities extends Department
class Philosophy extends Humanities
class Film extends Humanities

val cu_engineering = new Engineering()

val cu_csci = new CompSci()
val cu_mcen = new MechE()

val cu_phil = new Philosophy()
val cu_film = new Film()

defined class Department
defined class Engineering
defined class CompSci
defined class MechE
defined class Humanities
defined class Philosophy
defined class Film
cu_engineering: Engineering = $sess.cmd0Wrapper$Helper$Engineering@1e6716de
cu_csci: CompSci = $sess.cmd0Wrapper$Helper$CompSci@3e37373a
cu_mcen: MechE = $sess.cmd0Wrapper$Helper$MechE@43f901b3
cu_phil: Philosophy = $sess.cmd0Wrapper$Helper$Philosophy@324c39cd
cu_film: Film = $sess.cmd0Wrapper$Helper$Film@5ce7d007

## Variance
Variance is what we use to define how we want inheritance to work with generics (type parameters). It defines whether we want inheritance relationships to be preserved (covariant), reversed (contravariant), or ignored (invariant).

For example:
```scala
class Parent
class Child extends Parent

class MyNewClass[T]
```

Should `MyNewClass[Child]` inherit from `MyNewClass[Parent]`? Should it be reversed? should I not care?

We can say that a class is covariant with its parameters, or that it is contravariant with its parameters. Or that it's contravariant with some, covariant with others, and has no variance in the rest. It is an idea that  exists at the level of classes / types.

## Covariance
Covariance is the following idea: **If** class `A` inherits from class `B`, **then** `MyNewClass[A]` inherits from `MyNewClass[B]`. It states that you want to preserves the hierarchy. It moves in the same direction, hence the name **co**variant.

#### We will use this because we want:
```
ppl: comp sci course
prerequisite: engineering course = ppl
```

In [1]:
class Course[T <: Department](offering_department: T) {
    def get_department(): T = offering_department
}

val ppl = new Course(cu_csci)

// COMPILE ERROR: Didn't define variance
val prereq: Course[Engineering] = ppl

cmd1.sc:8: type mismatch;
 found   : Helper.this.Course[cmd1Wrapper.this.cmd0.wrapper.CompSci]
 required: Helper.this.Course[cmd1Wrapper.this.cmd0.wrapper.Engineering]
Note: cmd1Wrapper.this.cmd0.wrapper.CompSci <: cmd1Wrapper.this.cmd0.wrapper.Engineering, but class Course is invariant in type T.
You may wish to define T as +T instead. (SLS 4.5)
val prereq: Course[Engineering] = ppl
                                  ^

: 

In [2]:
class Course[+T <: Department](offering_department: T) {
    // We can *return* a `T` because we are covariant with `T`
    def get_department(): T = offering_department
    def has_best_TAs = false
}

val ppl = new Course(cu_csci) {
    override def has_best_TAs = true
}

// COMPILES!
val prereq: Course[Engineering] = ppl

defined class Course
ppl: Course[CompSci] = $sess.cmd1Wrapper$Helper$$anon$1@6dfbe55b
prereq: Course[Engineering] = $sess.cmd1Wrapper$Helper$$anon$1@6dfbe55b

### Note:
In the C# (C Sharp) language, covariance is defined with the `out` keyword, because functions are covariant with their output types. A function `(Int) => Engineering` can be replaced with `(Int) => CompSci` safely.

## Contravariance
**If** class `A` inherits from class `B`, **then** `MyNewClass[B]` inherits from `MyNewClass[A]` (note the swap). It flips the hierarchy. It moves in the oposite direction, hence the name **contra**variant.

#### We will use this because we want:
```
Bobby Braun: manager of engineering
cs manager: manager of computer science = Bobby Braun
```

In [2]:
class DepartmentHead[T <: Department](managed_department: T) {
    def manage(department: T) = println("You're fired!")
}

val bobby_braun = new DepartmentHead(cu_engineering)

// COMPILE ERROR: Didn't define variance
val cs_manager: DepartmentHead[CompSci] = bobby_braun

cmd2.sc:8: type mismatch;
 found   : Helper.this.DepartmentHead[cmd2Wrapper.this.cmd0.wrapper.Engineering]
 required: Helper.this.DepartmentHead[cmd2Wrapper.this.cmd0.wrapper.CompSci]
Note: cmd2Wrapper.this.cmd0.wrapper.Engineering >: cmd2Wrapper.this.cmd0.wrapper.CompSci, but class DepartmentHead is invariant in type T.
You may wish to define T as -T instead. (SLS 4.5)
val cs_manager: DepartmentHead[CompSci] = bobby_braun
                                          ^

: 

In [3]:
class DepartmentHead[-T <: Department](managed_department: T) {
    // We can *take* a `T` because we are contravariant with `T`
    def manage(department: T): Unit = println("You're fired!")
}

val bobby_braun = new DepartmentHead(cu_engineering)

// COMPILES!
val cs_manager: DepartmentHead[CompSci] = bobby_braun

defined class DepartmentHead
bobby_braun: DepartmentHead[Engineering] = $sess.cmd2Wrapper$Helper$DepartmentHead@26a3d71a
cs_manager: DepartmentHead[CompSci] = $sess.cmd2Wrapper$Helper$DepartmentHead@26a3d71a

### Note:
In the C# (C Sharp) language, contravariance is defined with the `in` keyword, because functions are contravariant with their input types. A function `(CompSci) => Int` can be replaced with `(Engineering) => Int` safely.

## Variance Exercises

For the following exercises, you can assume this code for the `Instructor` class:
```scala
class Instructor[+T <: Department](department: T)
```

### Exercises
1. Due to the chosen variance, we cannot add the following method to our `Course` class. Give an example of what would go wrong if Scala allowed adding methods like the one below (covariant type in input of method):
    ```scala
    def set_instructor(i: Instructor[T]): Unit
    ```
2. Due to the chosen variance, we cannot add the following method to our `DepartmentHead` class. Give an example of what would go wrong if Scala allowed adding methods like the one below (contravariant type in output of method):
    ```scala
    def get_list_of_taught_courses(): List[Course[T]]
    ```

3. If we made a class `ParkingMeter[PaymentMethod]` which takes in the specified type of payment, what would it's variance be?

4. If we made a class `MovieNight[Genre]` which plays a movie of the given genre, what would it's variance be?

5. If we made a class `Function[InType, OutType]`, should it be co- or contravariant in `InType`? `OutType`?

## Covariance with collections that allow insertion

#### We will use this because we want:
```scala
cs catalog = [ppl, algo, os]
me catalog = [sysd, thermo]
engineering catalog = cs catalog + me catalog
university catalog = engineering catalog + humanities catalog
```

In [4]:
class Catalog[T <: Department](val classes: List[Course[T]] = Nil) {
    def combine_with(other_catalog: Catalog[T]) =
        new Catalog(classes ++ other_catalog.classes)
}

val algo = new Course(cu_csci)
val os = new Course(cu_csci)

val cs_catalog: Catalog[CompSci] = new Catalog(List(ppl, algo))

defined class Catalog
algo: Course[CompSci] = $sess.cmd1Wrapper$Helper$Course@229d73d0
os: Course[CompSci] = $sess.cmd1Wrapper$Helper$Course@3e9585bc
cs_catalog: Catalog[CompSci] = $sess.cmd3Wrapper$Helper$Catalog@6ef4e667

In [4]:
val sysd = new Course(cu_mcen)
val thermo = new Course(cu_mcen)

val me_catalog: Catalog[MechE] = Catalog(List(sysd, thermo))

// COMPILE ERROR: no variance, lets add it.
val engineering_catalog: Catalog[Engineering] = cs_catalog.combine_with(me_catalog)

cmd4.sc:4: cmd4Wrapper.this.cmd3.wrapper.Catalog.type does not take parameters
val me_catalog: Catalog[MechE] = Catalog(List(sysd, thermo))
                                        ^cmd4.sc:7: type mismatch;
 found   : cmd4Wrapper.this.cmd3.wrapper.Catalog[cmd4Wrapper.this.cmd0.wrapper.MechE]
 required: cmd4Wrapper.this.cmd3.wrapper.Catalog[cmd4Wrapper.this.cmd3.cmd0.wrapper.CompSci]
val engineering_catalog: Catalog[Engineering] = cs_catalog.combine_with(me_catalog)
                                                                        ^

: 

### Note:
We want `MechE` and `CompSci` catalogs to count as engineering catalogs, so we want the `Catalog` class to be covariant. But we also want to add classes to it, meaning we need to put a covariant type into the parameter of a function, which isn't allowed!

In [4]:
class Catalog[+T <: Department](val classes: List[Course[T]] = Nil) {
    // COMPILE ERROR: Covariant type in parameter of method!
    def combine_with(other_catalog: Catalog[T]) =
        new Catalog(classes ++ other_catalog.classes)
}

cmd4.sc:3: covariant type T occurs in contravariant position in type Helper.this.Catalog[T] of value other_catalog
    def combine_with(other_catalog: Catalog[T]) =
                     ^

: 

### Note:
An alternative is to replace each `T` with the bound for `T`.

In [5]:
class Catalog[+T <: Department](val classes: List[Course[T]] = Nil) {
    // Potential solution:
    def combine_with(other_catalog: Catalog[Department]) =
        new Catalog(classes ++ other_catalog.classes)
}

defined class Catalog

In [6]:
val sysd = new Course(cu_mcen)
val thermo = new Course(cu_mcen)

val cs_catalog: Catalog[CompSci] = new Catalog(List(ppl, algo))
val me_catalog: Catalog[MechE] = new Catalog(List(sysd, thermo))

val engineering_catalog: Catalog[Department] = cs_catalog.combine_with(me_catalog)

sysd: Course[MechE] = $sess.cmd1Wrapper$Helper$Course@2125caf4
thermo: Course[MechE] = $sess.cmd1Wrapper$Helper$Course@1325f238
cs_catalog: Catalog[CompSci] = $sess.cmd4Wrapper$Helper$Catalog@4c2d92e4
me_catalog: Catalog[MechE] = $sess.cmd4Wrapper$Helper$Catalog@5d9d4eb9
engineering_catalog: Catalog[Department] = $sess.cmd4Wrapper$Helper$Catalog@7bc95782

### Note:
At first it looks like we've solved it, but now everytime we combine anything it becomes the very undescriptive `Catalog[Department]`. We know more than this. We know that if we run `cs_catalog.combine_with(me_catalog)`, we have a `Catalog[Engineering]`. Lets get the type system to agree with us.

In [6]:
// COMPILE ERROR: our typing bounds aren't as strong as we would like...
val engineering_catalog: Catalog[Engineering] = cs_catalog.combine_with(me_catalog)

cmd6.sc:1: type mismatch;
 found   : cmd6Wrapper.this.cmd5.cmd4.wrapper.Catalog[cmd6Wrapper.this.cmd5.cmd4.cmd0.wrapper.Department]
 required: cmd6Wrapper.this.cmd4.wrapper.Catalog[cmd6Wrapper.this.cmd0.wrapper.Engineering]
val engineering_catalog: Catalog[Engineering] = cs_catalog.combine_with(me_catalog)
                                                                       ^

: 

In [6]:
// DEFINE Catalog class here:
???

val cs_catalog: Catalog[CompSci] = new Catalog(List(ppl, algo))
val me_catalog: Catalog[MechE] = new Catalog(List(sysd, thermo))

val engineering_catalog: Catalog[Engineering] = cs_catalog.combine_with(me_catalog)

cmd6.sc:6: type mismatch;
 found   : cmd6Wrapper.this.cmd4.wrapper.Catalog[cmd6Wrapper.this.cmd4.cmd0.wrapper.Department]
 required: cmd6Wrapper.this.cmd4.wrapper.Catalog[cmd6Wrapper.this.cmd0.wrapper.Engineering]
val engineering_catalog: Catalog[Engineering] = cs_catalog.combine_with(me_catalog)
                                                                       ^

: 